In [1]:
import os
from pathlib import Path

import pandas as pd
import numpy as np

from shapely import wkt
from shapely.geometry import MultiPolygon

import pandas as pd
import numpy as np
from shapely import wkt
from shapely.geometry import MultiPolygon
from collections import defaultdict
import json

import utils.energyplus_utils as uep


In [2]:
current_directory = Path.cwd()
version_n = '90'

data_folder = current_directory.parents[0] / 'data'

csv_folder = current_directory / 'outputs' / 'jsons'
csv_file = csv_folder / f'wkt_v{version_n}.csv'
print(csv_file)
df = pd.read_csv(csv_file)

    

c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\script\outputs\jsons\wkt_v90.csv


EXTRACT GEOMETRY

In [3]:
extract_geometries_from_csv = True

if extract_geometries_from_csv:

    geometry_df = df[df['geometry_wkt'].notnull()].copy()
    geometry_df["vertices"] = geometry_df["geometry_wkt"].apply(uep.extract_vertices)

    geometry_df_valid = geometry_df[pd.to_numeric(geometry_df["parent_id"], errors="coerce").notnull()].copy()
    geometry_df_valid["parent_id"] = pd.to_numeric(geometry_df_valid["parent_id"], errors="coerce").astype(int)

    rooms_df = df[(df["thematic_surface"] == "room") & (df["element_id"].notnull())]
    valid_rooms_df = rooms_df[pd.to_numeric(rooms_df["element_id"], errors="coerce").notnull()].copy()
    valid_rooms_df["element_id"] = pd.to_numeric(valid_rooms_df["element_id"], errors="coerce").astype(int)

    room_id_to_surfaces = {
        room_id: geometry_df_valid[geometry_df_valid["parent_id"] == room_id]
        for room_id in valid_rooms_df["element_id"]
    }

geometry_df_valid.head(50)


,csv_line_id,element_id,thematic_surface,parent_id,geometry_wkt,element_gml_id,has_holes,related_to,vertices
2,3,2,partly_wall,1,"MULTIPOLYGON Z (((-39.014 22.381 6.657, -39.11...",8041e176-bbed-5c89-93c2-a06fbf790508,False,"[0, 1]","[[(-39.014, 22.381, 6.657), (-39.119, 29.884, ..."
3,4,3,external,1,"MULTIPOLYGON Z (((-39.119 29.884 6.657, -31.66...",94b31277-a223-5a14-a9b3-17ee03a8e96c,False,1,"[[(-39.119, 29.884, 6.657), (-31.666, 29.983, ..."
4,5,3,external_with_hole,1,MULTIPOLYGON Z (((-31.666 29.983000000000004 6...,94b31277-a223-5a14-a9b3-17ee03a8e96c,True,[],"[[(-31.666, 29.983000000000004, 6.657), (-31.6..."
5,6,5,external,1,"MULTIPOLYGON Z (((-31.666 29.983 3.052, -31.66...",83117968-e38e-51f4-a1c4-c7444f1bb879,False,1,"[[(-31.666, 29.983, 3.052), (-31.666, 29.983, ..."
6,7,6,external,1,"MULTIPOLYGON Z (((-31.513 22.966 3.052, -31.51...",589667fd-26c2-569f-a79c-03d0754269e0,False,1,"[[(-31.513, 22.966, 3.052), (-31.513, 22.966, ..."
7,8,7,partly_wall,1,"MULTIPOLYGON Z (((-31.502 22.488 6.657, -39.01...",0910b8a4-e496-5232-ab39-8ef8b4548602,False,"[1, 2]","[[(-31.502, 22.488, 6.657), (-39.014, 22.381, ..."
8,9,7,partly_wall_with_hole,1,"MULTIPOLYGON Z (((-39.014 22.381 6.657, -31.50...",0910b8a4-e496-5232-ab39-8ef8b4548602,True,"[1, 2]","[[(-39.014, 22.381, 6.657), (-31.502, 22.488, ..."
9,10,10,ceiling,1,"POLYGON Z ((-39.014 22.381 6.657, -31.502 22.4...",4e6cfbb7-2f45-5ef5-89a5-f31bb4d5fa5d,False,[],[]
10,11,9,floor,1,"POLYGON Z ((-39.014 22.381 3.052, -39.119 29.8...",dbec5950-1a55-56c5-abb5-e7c01fce407f,False,[],[]
11,12,4,window,3,"MULTIPOLYGON Z (((-38.97 29.886 4.519, -38.97 ...",f4244607-679d-5dd7-a77e-e8e6089f66e9,True,[],"[[(-38.97, 29.886, 4.519), (-38.97, 29.886, 5...."


EXTRACT MATERIALS AND CONSTRUCTIONS

In [4]:
import_materials_and_constructions = True

if import_materials_and_constructions:

    df = pd.read_csv(csv_file)

    current_directory = os.getcwd()
    json_materials = os.path.join(current_directory, 'IDL_files', 'materials_constructions', 'materials.json')  
    json_constructions = os.path.join(current_directory, 'IDL_files', 'materials_constructions', 'constructions.json')

    with open(json_materials, 'r') as f:
        materials_library = json.load(f)
    
    with open(json_constructions, 'r') as m:
        constructions_library =  json.load(m)

    df_materials = pd.DataFrame(materials_library)
    df_constructions = pd.DataFrame(constructions_library)
    import uuid

    df_constructions = df_constructions.copy()
    df_constructions['construction_uuid'] = [str(uuid.uuid4()) for _ in range(len(df_constructions))]

# df_materials.head()
df_constructions.head(80)


,name,wall_type,match_thickness,layers,construction_uuid
0,external_wall_brick_10cm,external_wall,0.125,"[{'material': 'plaster_external', 'material_uu...",7bc4f3b2-01ff-492d-b066-df9fb349adf5
1,external_wall_brick_12cm,external_wall,0.150,"[{'material': 'plaster_external', 'material_uu...",c89e5345-5d73-4978-93cf-3727f2e28ca3
2,external_wall_brick_15cm,external_wall,0.175,"[{'material': 'plaster_external', 'material_uu...",2591dbb0-5be7-4645-83fc-6e1a6bce1bfc
3,external_wall_brick_17cm,external_wall,0.200,"[{'material': 'plaster_external', 'material_uu...",52c6fc70-62af-406a-bf1c-abaf45401b4a
4,external_wall_brick_20cm,external_wall,0.225,"[{'material': 'plaster_external', 'material_uu...",8343a281-a55d-4bb7-880c-7af227888107
...,...,...,...,...,...
75,external_wall_concrete_15cm_insul_150mm,external_wall,0.325,"[{'material': 'plaster_external', 'material_uu...",ad7d286e-b318-4cbc-a0d6-68f10304a4cd
76,external_wall_concrete_15cm_insul_200mm,external_wall,0.375,"[{'material': 'plaster_external', 'material_uu...",ba8b20b7-0bf3-4827-8d31-35e3625d8718
77,external_wall_concrete_17cm,external_wall,0.200,"[{'material': 'plaster_external', 'material_uu...",92b7ca50-6952-4d1d-9b6b-4ff33571b6d2
78,external_wall_concrete_17cm_insul_25mm,external_wall,0.225,"[{'material': 'plaster_external', 'material_uu...",07471766-30fa-4a27-a938-a44ee4877e5c


In [ ]:
from collections import defaultdict
import json

epjson = defaultdict(dict)

# --- Version ---
epjson["Version"]["Version 1"] = {"version_identifier": "9.6"}

# --- Material and Construction ---
epjson["Material"] = {
    "DefaultWallMaterial": {
        "roughness": "MediumSmooth",
        "thickness": 0.2,
        "conductivity": 0.5,
        "density": 800,
        "specific_heat": 1000
    }
}
epjson["Construction"] = {
    "DefaultWall": {
        "layers": ["DefaultWallMaterial"]
    }
}

# --- Required Metadata ---
epjson["SimulationControl"] = {
    "SimulationControl 1": {
        "do_zone_sizing_calculation": False,
        "do_system_sizing_calculation": False,
        "do_plant_sizing_calculation": False,
        "run_simulation_for_sizing_periods": False,
        "run_simulation_for_weather_file_run_periods": True
    }
}
epjson["Timestep"]["Timestep 1"] = {"number_of_timesteps_per_hour": 6}
epjson["RunPeriod"]["RunPeriod 1"] = {
    "name": "Annual",
    "begin_month": 1,
    "begin_day_of_month": 1,
    "end_month": 12,
    "end_day_of_month": 31,
    "use_weather_file_daylight_saving_period": True,
    "use_weather_file_holidays_and_special_days": True
}
epjson["Output:SQLite"] = {"Output:SQLite 1": {}}
epjson["Output:Variable"] = {
    "Zone Temp Output": {
        "key_value": "*",
        "variable_name": "Zone Mean Air Temperature",
        "reporting_frequency": "Hourly"
    }
}

# --- Generate Zones and Surfaces ---
for room_id, surfaces in room_id_to_surfaces.items():
    zone_name = f"Zone_{room_id}"
    epjson["Zone"][zone_name] = {
        "name": zone_name,
        "direction_of_relative_north": 0.0,
        "x_origin": 0.0,
        "y_origin": 0.0,
        "z_origin": 0.0,
        "type": 1,
        "multiplier": 1,
        "ceiling_height": "Autocalculate",
        "volume": "Autocalculate"
    }

    for idx, row in surfaces.iterrows():
        surface_type = uep.get_surface_type(row["thematic_surface"])
        if surface_type == "Window":  # Skip invalid surface_type in BuildingSurface:Detailed
            continue
        if not isinstance(row["vertices"], list) or len(row["vertices"]) == 0 or len(row["vertices"][0]) < 3:
            continue

        surface_name = f"{zone_name}_{surface_type}_{idx}"
        coords = row["vertices"][0]
        surface_data = {
            "name": surface_name,
            "surface_type": surface_type,
            "construction_name": "DefaultWall",
            "zone_name": zone_name,
            "outside_boundary_condition": "Outdoors",
            "sun_exposure": "SunExposed",
            "wind_exposure": "WindExposed",
            "view_factor_to_ground": 0.5,
            "number_of_vertices": len(coords)
        }
        for i, (x, y, z) in enumerate(coords):
            surface_data[f"vertex_{i+1}_x_coordinate"] = round(float(x), 3)
            surface_data[f"vertex_{i+1}_y_coordinate"] = round(float(y), 3)
            surface_data[f"vertex_{i+1}_z_coordinate"] = round(float(z), 3)

        epjson["BuildingSurface:Detailed"][surface_name] = surface_data

# --- Preview Output ---
sample_zone_key = list(epjson["Zone"].keys())[0]
sample_surface_keys = list(epjson["BuildingSurface:Detailed"].keys())[:3]
epjson_preview = {
    "Zone": {sample_zone_key: epjson["Zone"][sample_zone_key]},
    "BuildingSurface:Detailed": {k: epjson["BuildingSurface:Detailed"][k] for k in sample_surface_keys}
}

print(json.dumps(epjson_preview, indent=2)[:3000])


'{\n  "Zone": {\n    "Zone_1": {\n      "name": "Zone_1",\n      "direction_of_relative_north": 0.0,\n      "x_origin": 0.0,\n      "y_origin": 0.0,\n      "z_origin": 0.0,\n      "type": 1,\n      "multiplier": 1,\n      "ceiling_height": "autocalculate",\n      "volume": "autocalculate"\n    }\n  },\n  "BuildingSurface:Detailed": {\n    "Zone_1_Wall_2": {\n      "name": "Zone_1_Wall_2",\n      "surface_type": "Wall",\n      "construction_name": "DefaultWall",\n      "zone_name": "Zone_1",\n      "outside_boundary_condition": "Outdoors",\n      "sun_exposure": "SunExposed",\n      "wind_exposure": "WindExposed",\n      "view_factor_to_ground": 0.5,\n      "number_of_vertices": 5,\n      "vertex_1_x_coordinate": -39.014,\n      "vertex_1_y_coordinate": 22.381,\n      "vertex_1_z_coordinate": 6.657,\n      "vertex_2_x_coordinate": -39.119,\n      "vertex_2_y_coordinate": 29.884,\n      "vertex_2_z_coordinate": 6.657,\n      "vertex_3_x_coordinate": -39.119,\n      "vertex_3_y_coordinate

ALL ZONES

In [ ]:
import pandas as pd
import json
from collections import defaultdict

# --- INPUT: assume geometry_df already exists and includes 'vertices' column --- #
# Example expected columns: 
# ['csv_line_id', 'element_id', 'thematic_surface', 'parent_id', 
#  'geometry_wkt', 'element_gml_id', 'has_holes', 'related_to', 'vertices']

# --- Step 1: Clean and filter valid geometry --- #
geometry_df_valid = geometry_df[pd.to_numeric(geometry_df["parent_id"], errors="coerce").notnull()].copy()
geometry_df_valid["parent_id"] = pd.to_numeric(geometry_df_valid["parent_id"], errors="coerce").astype(int)

# --- Step 2: Group surfaces by room (zone) --- #
valid_room_ids = geometry_df_valid["parent_id"].unique()
room_id_to_surfaces = {
    room_id: geometry_df_valid[geometry_df_valid["parent_id"] == room_id]
    for room_id in valid_room_ids
}

# --- Step 3: Start EPJSON structure --- #
epjson = defaultdict(dict)

# Metadata
epjson["Version"]["Version 1"] = {"version_identifier": "9.6"}
epjson["SimulationControl"]["SimulationControl 1"] = {
    "do_zone_sizing_calculation": False,
    "do_system_sizing_calculation": False,
    "do_plant_sizing_calculation": False,
    "run_simulation_for_sizing_periods": False,
    "run_simulation_for_weather_file_run_periods": True
}
epjson["Timestep"]["Timestep 1"] = {"number_of_timesteps_per_hour": 6}
epjson["RunPeriod"]["RunPeriod 1"] = {
    "name": "Annual",
    "begin_month": 1,
    "begin_day_of_month": 1,
    "end_month": 12,
    "end_day_of_month": 31,
    "use_weather_file_daylight_saving_period": True,
    "use_weather_file_holidays_and_special_days": True
}
epjson["Output:Variable"]["Zone Temp Output"] = {
    "key_value": "*",
    "variable_name": "Zone Mean Air Temperature",
    "reporting_frequency": "Hourly"
}
epjson["Output:SQLite"]["Output:SQLite 1"] = {}

# --- Step 4: Default material and construction --- #
epjson["Material"]["DefaultWallMaterial"] = {
    "roughness": "MediumSmooth",
    "thickness": 0.2,
    "conductivity": 0.5,
    "density": 800,
    "specific_heat": 1000
}
epjson["Construction"]["DefaultWall"] = {
    "layers": ["DefaultWallMaterial"]
}

# --- Step 6: Generate zones and surfaces --- #
for room_id, surfaces in room_id_to_surfaces.items():
    zone_name = f"Zone_{room_id}"
    epjson["Zone"][zone_name] = {
        "name": zone_name,
        "direction_of_relative_north": 0.0,
        "x_origin": 0.0,
        "y_origin": 0.0,
        "z_origin": 0.0,
        "type": 1,
        "multiplier": 1,
        "ceiling_height": "Autocalculate",  # ✅ Correct case
        "volume": "Autocalculate"
    }

    for idx, row in surfaces.iterrows():
        surface_type = uep.get_surface_type(row["thematic_surface"])
        if surface_type is None:
            continue  # Skip windows

        if not isinstance(row["vertices"], list) or len(row["vertices"]) == 0:
            continue
        coords = row["vertices"][0]
        if len(coords) < 3:
            continue  # Skip bad geometry

        surface_name = f"{zone_name}_{surface_type}_{idx}"
        surface_data = {
            "name": surface_name,
            "surface_type": surface_type,
            "construction_name": "DefaultWall",
            "zone_name": zone_name,
            "outside_boundary_condition": "Outdoors",
            "sun_exposure": "SunExposed",
            "wind_exposure": "WindExposed",
            "view_factor_to_ground": 0.5,
            "number_of_vertices": len(coords)
        }

        for i, (x, y, z) in enumerate(coords):
            surface_data[f"vertex_{i+1}_x_coordinate"] = round(float(x), 3)
            surface_data[f"vertex_{i+1}_y_coordinate"] = round(float(y), 3)
            surface_data[f"vertex_{i+1}_z_coordinate"] = round(float(z), 3)

        epjson["BuildingSurface:Detailed"][surface_name] = surface_data

# --- Step 7: Export EPJSON to file --- #
with open("output.epJSON", "w") as f:
    json.dump(epjson, f, indent=2)



In [8]:
with open("output.epJSON", "w") as f:
    json.dump(epjson, f, indent=2)

In [7]:
ssssss

NameError: name 'ssssss' is not defined

In [ ]:
import_materials_and_constructions = True

if import_materials_and_constructions:

    df = pd.read_csv(csv_file)

    current_directory = os.getcwd()
    json_materials = os.path.join(current_directory, 'IDL_files', 'materials_constructions', 'materials.json')  
    json_constructions = os.path.join(current_directory, 'IDL_files', 'materials_constructions', 'constructions.json')

    with open(json_materials, 'r') as f:
        materials_library = json.load(f)
    
    with open(json_constructions, 'r') as m:
        constructions_library =  json.load(m)

    df_materials = pd.DataFrame(materials_library)
    df_constructions = pd.DataFrame(constructions_library)
    import uuid

    df_constructions = df_constructions.copy()
    df_constructions['construction_uuid'] = [str(uuid.uuid4()) for _ in range(len(df_constructions))]

df_materials.head()
# df_constructions.head(80)



,uuid,material_id,name,lambda_W_per_mK,density_kg_per_m3,g_value
0,b0a6a2d1-1001-4ff6-a7f9-1b98c7b1f001,101,plaster_internal,0.70,950,NaN
1,b0a6a2d1-1002-4ff6-a7f9-1b98c7b1f002,102,bricks_full,0.72,1800,NaN
2,b0a6a2d1-1003-4ff6-a7f9-1b98c7b1f003,103,bricks_perforated,0.45,1400,NaN
3,b0a6a2d1-1004-4ff6-a7f9-1b98c7b1f004,104,concrete_reinforced,2.10,2400,NaN
4,b0a6a2d1-1005-4ff6-a7f9-1b98c7b1f005,105,concrete_lightweight,0.60,1100,NaN


In [ ]:
import ast

unique_materials = set()
unique_constructions = set(df_materials["construction_name"].unique())

# Parse each construction layer list and extract material names
for layers in df_materials["construction_layers"]:
    try:
        layer_list = ast.literal_eval(layers)
        for layer in layer_list:
            unique_materials.add(layer["material"])
    except:
        continue

# Prepare the summary
material_construction_summary = {
    "Total Unique Materials": len(unique_materials),
    "Total Unique Constructions": len(unique_constructions),
    "Sample Materials": list(unique_materials)[:5],
    "Sample Constructions": list(unique_constructions)[:5]
}

material_construction_summary


KeyError: 'construction_name'

In [ ]:
import ast

# Initialize storage for constructions and materials
materials_dict = {}
constructions_dict = {}

# Process each unique construction
for _, row in df_materials.iterrows():
    construction_name = row["construction_name"]
    try:
        layers = ast.literal_eval(row["construction_layers"])
    except Exception:
        continue  # skip if the construction_layers field is invalid

    material_names = []
    for layer in layers:
        mat_name = layer["material"]
        material_names.append(mat_name)

        # Add placeholder Material if not already added
        if mat_name not in materials_dict:
            materials_dict[mat_name] = {
                "roughness": "MediumSmooth",
                "thickness": 0.1,           # Placeholder; can be refined
                "conductivity": 0.5,
                "density": 800,
                "specific_heat": 1000
            }

    # Add Construction entry
    if construction_name not in constructions_dict:
        constructions_dict[construction_name] = {
            "layers": material_names
        }

# Update EPJSON structure with real materials and constructions
epjson["Material"].update(materials_dict)
epjson["Construction"].update(constructions_dict)

# Build mapping from element_id to construction_name
element_to_construction = dict(
    zip(df_materials["element_id"], df_materials["construction_name"])
)

# Apply construction names to BuildingSurface:Detailed entries
for surface_name, surface in epjson["BuildingSurface:Detailed"].items():
    zone_name = surface["zone_name"]
    room_id = int(zone_name.split("_")[1])
    element_id = int(surface_name.split("_")[-1])
    construction_name = element_to_construction.get(element_id)
    if construction_name:
        surface["construction_name"] = construction_name

# Show a small sample of materials and constructions for confirmation
preview = {
    "Construction": dict(list(epjson["Construction"].items())[:2]),
    "Material": dict(list(epjson["Material"].items())[:3])
}
json.dumps(preview, indent=2)




KeyError: 'construction_name'